In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.computation import RPComputation
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from tqdm import tqdm

In [2]:
# Función para calcular la información mutua para varios retrasos temporales
def mutual_information_delay(series1, series2, max_lag):
    mi = []
    for lag in range(1, max_lag + 1):
        x = series1[:-lag]
        y = series2[lag:]
        if len(x) > 3 and len (y) > 3:
            mi.append(mutual_info_regression(x.reshape(-1, 1), y)[0])
    return mi

In [3]:
def calculate_entropy(matrix):
    probabilities = np.histogram(matrix, bins=256, density=True)[0]
    return entropy(probabilities)

In [4]:
def calculate_recurrence_plot(signal, optimal_delay, desired_rr):
    # Step 1: Create the TimeSeries object
    time_series = TimeSeries(signal,
                             embedding_dimension=1,
                             time_delay=optimal_delay)
    
    # Step 2: Define initial settings
    initial_radius = 0.5  # Start with a small radius
    radius_step = 0.1  # Small step size for finer adjustment
    max_iterations = 100  # Max number of iterations to avoid infinite loop
    tolerance = 0.01  # Tolerance for recurrence rate difference

    # Function to calculate recurrence rate
    def compute_rr(radius):
        settings = Settings(time_series,
                            analysis_type=Classic,
                            similarity_measure=EuclideanMetric,
                            neighbourhood=FixedRadius(radius))
        computation = RPComputation.create(settings)
        rp = computation.run()
        recurrence_matrix = rp.recurrence_matrix_reverse
        rr = (recurrence_matrix.sum() - len(recurrence_matrix)) / (recurrence_matrix.size - len(recurrence_matrix))
        return rr, recurrence_matrix

    # Step 3: Iteratively adjust the radius to match the desired recurrence rate
    current_rr, recurrence_matrix = compute_rr(initial_radius)
    iteration = 0
    while abs(current_rr - desired_rr) > tolerance and iteration < max_iterations:
        if current_rr < desired_rr:
            initial_radius /= (1-radius_step)
        else:
            initial_radius *= (1-radius_step)
        current_rr, recurrence_matrix = compute_rr(initial_radius)
        #print(current_rr, initial_radius)
        iteration += 1

    # Return the recurrence matrix with the desired recurrence rate
    return recurrence_matrix

In [5]:
df_active_value = pd.read_csv('../data/redd_active_value_f1hz.csv')

In [6]:
attributes = [c for c in df_active_value.columns.values if not c in ['timestamp']]
labels = [c for c in df_active_value.columns.values if not c in ['timestamp', 'mains', 'amplitude_spectrum', 'phase_spectrum']]
#labels = ['Microwave01','Washer dryer01','Washer dryer02']
predictors = ['mains', 'amplitude_spectrum', 'phase_spectrum']
index_name = 'timestamp'
training_start = '2011-04-16'
training_end = '2011-05-16'
test_start = '2011-05-17'
test_end = '2011-05-31'

window_sizes = [224]
number_of_tests = 15
recurrence_rates = [0.1, 0.20, 0.30, 0.40, 0.50]

In [7]:
labels

['Microwave01', 'Washer dryer01', 'Washer dryer02']

In [8]:
# Ensure 'timestamp' column is in datetime format
df_active_value[index_name] = pd.to_datetime(df_active_value[index_name])
# Set the index as the timestamp
df_active_value.set_index(index_name, inplace=True)

In [9]:
# Partition the DataFrame into training and test sets
training_active_value_set = df_active_value.loc[training_start:training_end]
test_active_value_set = df_active_value.loc[test_start:test_end]


In [10]:
training_active_value_set.shape, test_active_value_set.shape

((2674113, 23), (1196394, 23))

In [11]:
scaler = StandardScaler()
scaler = scaler.fit(training_active_value_set)
training_active_value_set = scaler.transform(training_active_value_set)
training_active_value_set = pd.DataFrame(training_active_value_set)
training_active_value_set.columns = attributes

In [12]:
result_list = list()
for label in labels:
    for predictor in predictors:
        for window_size in window_sizes:
            rr_result = dict()
            rr_result['label'] = label
            rr_result['predictor'] = predictor
            print(rr_result)
            # calculate best time_delay based in predictor vs label
            time_step = int(training_active_value_set.shape[0] / number_of_tests)
            i, d = divmod(training_active_value_set.shape[0] - window_size + 1, time_step)
            row_factor = i + 1
            optimal_delay_list = list()
            for r in tqdm(range(int(row_factor))):
                if r * time_step >= window_size:
                    signal_1 = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [predictor]].values
                    signal_2 = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [label]].values
                    mi_values = mutual_information_delay(signal_1.ravel(), # type: ignore
                                                        signal_2.ravel(), int(window_size/2)) 
                    optimal_delay = np.argmax(mi_values) + 1
                    optimal_delay_list.append(optimal_delay)
            optimal_delay_median = int(np.median(optimal_delay_list))

            rr_result['windows_size'] = window_size
            rr_result['optimal_delay'] = optimal_delay_median 
            print(rr_result)           
            # calculate best recurrence rate based predictor
            for rr in recurrence_rates:
                rr_result['rr'] = rr                
                coef1_list = list()
                coef2_list = list()
                coef3_list = list()
                for r in tqdm(range(int(row_factor))):
                    if r * time_step >= window_size:
                        signal = training_active_value_set.loc[r * time_step - window_size:r * time_step-1, [predictor]].values                        
                        # Obtener la matriz de recurrencia
                        recurrence_matrix = calculate_recurrence_plot(signal, optimal_delay_median, rr) # type: ignore

                        # Calcular la entropía y la PSVS
                        ent = calculate_entropy(recurrence_matrix) # type: ignore
                        U, S, Vt = np.linalg.svd(recurrence_matrix)
                        svs = np.empty(len(S), dtype=np.float32)
                        for i in range(len(S)):
                            svs[i] = math.log((S[i] / (sum(S) + 0.000000001)) + 0.000000001)
                        SVS_prin_cond = (np.max(svs) + np.min(svs)) / 2
                        PSVS = 0.0
                        for i in range(len(S)):
                            if svs[i] >= SVS_prin_cond:
                                PSVS += 1
                        PSVS /= len(S)

                        # Calcular la relación entre la entropía y la SVS
                        coef1 = math.sqrt(ent) * PSVS * PSVS 

                        # Calcular la relación entre la entropía y la SVS
                        coef2 = math.sqrt(ent) * PSVS 

                        # Calcular la relación entre la entropía y la SVS
                        coef3 = ent * PSVS

                        # Add to results of coefs
                        coef1_list.append(coef1)
                        coef2_list.append(coef2)
                        coef3_list.append(coef3)
            
                rr_result['sqrt_ent_psvs_psvs'] = np.mean(coef1_list)
                rr_result['sqrt_ent_psvs'] = np.mean(coef2_list)
                rr_result['ent_psvs'] = np.mean(coef3_list)     
                print(rr_result)       
                result_list.append(rr_result)
                

                        

            


{'label': 'Microwave01', 'predictor': 'mains'}


100%|██████████| 15/15 [00:03<00:00,  4.83it/s]


{'label': 'Microwave01', 'predictor': 'mains', 'windows_size': 224, 'optimal_delay': 81}


  0%|          | 0/15 [00:00<?, ?it/s]c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyopencl\invoker.py:376: UserWarning: pytools.persistent_dict 'pyopencl-invoker-cache-v41': enabling safe_sync as default. This provides strong protection against data loss, but can be unnecessarily expensive for use cases such as caches.Pass 'safe_sync=False' if occasional data loss is tolerable. Pass 'safe_sync=True' to suppress this warning.
  invoker_cache = WriteOncePersistentDict(
100%|██████████| 15/15 [01:31<00:00,  6.07s/it]


{'label': 'Microwave01', 'predictor': 'mains', 'windows_size': 224, 'optimal_delay': 81, 'rr': 0.1, 'sqrt_ent_psvs_psvs': 0.07552611765338114, 'sqrt_ent_psvs': 0.1370281364908585, 'ent_psvs': 0.08077875826827079}


 93%|█████████▎| 14/15 [01:27<00:06,  6.28s/it]
c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyrqa\opencl.py:196: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if native_vector_width is 0:
c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyrqa\opencl.py:196: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if native_vector_width is 0:
c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyrqa\opencl.py:196: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if native_vector_width is 0:
c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyrqa\opencl.py:196: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if native_vector_width is 0:
c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyrqa\opencl.py:196: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if native_vector_width is 0:
c:\Users\denny\.conda\envs\nilmtk-env\lib\site-packages\pyrqa\opencl.py:196: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if 

KeyboardInterrupt: 